# **Práctica 3**: GANs
# Parte 1-2: GANs simples - Ejercicios Exttas

Simplificación de:
https://medium.com/@mattiaspinelli/simple-generative-adversarial-network-gans-with-keras-1fe578e44a87


### Ejercicio EXTRA:
Modifica el código para balancear cuanto aprende el generador y cuanto el discriminador.

Por ejemplo haz que el número de datos en el batch de entrenamiento dependa del coste (loss) en el paso de entrenamiento anterior de cada modelo (generador o discriminador). Es decir, si el discriminador aprende mucho (su loss es bajo) en el siguiente paso de entrenamiento le daré menos datos (opara que aprenda menos).  

1. Definimos las variables "gen_batch_size" y "disc_batch_size" que serán las que determinen el tamaño del batch de cada modelo en cada iteración del bucle de entrenamiento. Establecemos su valor inicial en "batch", el tamaño de batch original.

3. En cada iteración del bucle de entrenamiento, calculamos el coste (loss) de cada modelo en la iteración anterior y lo almacenamos en las variables "d_loss_prev" y "g_loss_prev".

3. Para el discriminador, comparamos su coste anterior con el coste actual y, si ha mejorado, reducimos el tamaño del batch en un 10%, mientras que si ha empeorado, lo aumentamos en un 10%. En cualquier caso, aseguramos que el tamaño mínimo del batch sea 1.

4. Para el generador, hacemos lo mismo que para el discriminador pero con la inversa, es decir, si ha mejorado aumentamos el tamaño del batch y si ha empeorado lo reducimos.

5. Utilizamos los tamaños de batch calculados para entrenar los modelos en cada iteración.

In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.layers import Input, Dense, Reshape, Flatten, Dropout
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers.legacy import Adam

import matplotlib.pyplot as plt



#from IPython.core.debugger import Tracer

from PIL import Image

In [ ]:
#tf.config.run_functions_eagerly(True)
# Set random seed
#tf.random.set_seed (42)

In [ ]:
# Datos
(X_train, _), (_, _) = mnist.load_data()

# Rescale -1 to 1
X_train = (X_train.astype(np.float32) - 127.5) / 127.5
X_train = np.expand_dims(X_train, axis=3)

In [ ]:
width=28
height=28
channels=1

in_shape = X_train.shape
in_shape = in_shape[1:]
OPTIMZADOR_ADAM = Adam(learning_rate=0.0002, beta_1=0.5, decay=8e-8)

In [ ]:
model_gen = Sequential()
model_gen.add (Dense (256, input_shape = (100,)))
model_gen.add (LeakyReLU (alpha = 0.2))
model_gen.add (BatchNormalization (momentum = 0.8))
model_gen.add (Dense (512))
model_gen.add (LeakyReLU (alpha = 0.2))
model_gen.add (BatchNormalization (momentum = 0.8))
model_gen.add (Dense(1024))
model_gen.add (LeakyReLU (alpha=0.2))
model_gen.add (BatchNormalization (momentum = 0.8))
model_gen.add (Dense (np.prod (in_shape), activation = 'tanh'))
model_gen.add (Reshape (in_shape))
model_gen.summary ()
model_gen.compile (loss='binary_crossentropy', optimizer=OPTIMZADOR_ADAM)

model_Disc = Sequential ()
model_Disc.add (Flatten(input_shape = in_shape))
model_Disc.add (Dense(128, input_shape = in_shape))
model_Disc.add (LeakyReLU(alpha = 0.2))
model_Disc.add (Dense(64))
model_Disc.add (LeakyReLU(alpha = 0.2))
model_Disc.add (Dense(1, activation = 'sigmoid'))
 
model_Disc.compile (loss='binary_crossentropy', optimizer=OPTIMZADOR_ADAM, metrics = ['accuracy'])

model_gan = Sequential()
model_gan.add (model_gen)
model_gan.add (model_Disc)
model_gan.compile (loss='binary_crossentropy', optimizer=OPTIMZADOR_ADAM, run_eagerly = True)


model_gan.summary()

In [ ]:
# Parámetros del entrenamiento
epochs = 5000
batch = 10
gen_batch_size  = batch
disc_batch_size = batch/2

sample_interval = 100
num_classes     = 10
latent_dim      = 100
d_loss_prev     = np.inf
g_loss_prev     = np.inf

DD_loss = np.zeros((epochs,))
GG_loss = np.zeros((epochs,))


for cnt in range(epochs):

    ## Entrenamos discriminador
        # Imágenes reales
    random_index = np.random.randint(0, len(X_train) - np.int64(disc_batch_size))
    legit_images = X_train[random_index : random_index + np.int64(disc_batch_size)].reshape((np.int64(disc_batch_size),)+in_shape)
        # Imágenes sintéticas
    gen_noise = np.random.normal(0, 1, (np.int64(disc_batch_size), 100))
    print('gen_noise:',gen_noise.shape)
    syntetic_images = model_gen.predict(gen_noise)
        # Combinamos imágenes reales y sintéticas
    x_combined_batch = np.concatenate((legit_images, syntetic_images))
    y_combined_batch = np.concatenate((np.ones((np.int64(disc_batch_size), 1)), np.zeros((np.int64(disc_batch_size), 1))))
        # Entrenamos discriminador
    d_loss = model_Disc.train_on_batch(x_combined_batch, y_combined_batch)
   
    # Calculamos el cambio en la loss del discriminador
    print('d_loss_prev:',d_loss_prev)
    print('d_loss[0]:',d_loss[0])
    d_loss_change = d_loss_prev - d_loss[0]
    d_loss_prev = d_loss[0]
    if d_loss_change > 0:
        disc_batch_size = min(max(1, int(disc_batch_size * 0.9)),128)
        print('disc_batch_size:max(1, int(disc_batch_size * 0.9)')
    else:
        disc_batch_size = min(int(disc_batch_size * 1.50),128)
        print('disc_batch_size:int(disc_batch_size * 1.1)')
        
    print('disc_batch_size:',disc_batch_size)
    ## Entrenamos generador
        # Imágenes sintéticas
    noise = np.random.normal(0, 1, (gen_batch_size, 100))
    y_mislabled = np.ones((gen_batch_size, 1))
        # Entremaos generador
    g_loss = model_gan.train_on_batch(noise, y_mislabled)
    ## Ajustamos tamaño de batch para generador
    if g_loss < g_loss_prev:
        gen_batch_size = min(int(gen_batch_size * 1.50),128)
    else:
        gen_batch_size = min(max(1, int(gen_batch_size * 0.9)),128)
    g_loss_prev = g_loss
    ## Evolución entrenamiento
    print ('epoch: %d, [Discriminator :: d_loss: %f], [ Generator :: loss: %f]' % (cnt, d_loss[0], g_loss))

    DD_loss[cnt] = d_loss[0]
    GG_loss[cnt] = g_loss

    if cnt % sample_interval == 0:

        # Generar imágenes falsas a partir de vectores de ruido aleatorios
        noise = np.random.normal(0, 1, (num_classes, latent_dim))
        images = model_gen.predict(noise)
        print('images:',images.shape)
             
        images = 0.5 * images + 0.5 # escalar de [-1,1] a [0,1]
        #images = np.uint8(255 * images) # escalar a [0, 255]
        # Mostrar las imágenes generadas
        r = 2
        c = 5
        fig, axs = plt.subplots(r, c,figsize=(11,6.9),dpi=100)
        count = 0
        for i in range(r):
            for j in range(c):
                axs[i,j].imshow(images[count, :, :,0],cmap='gray') #, cmap='gray'
                axs[i,j].axis('off')
                count += 1
        plt.show()
    #batch_D = int(batch/(d_loss[0]+1e-8))

In [ ]:
%matplotlib inline 
plt.figure
plt.plot (GG_loss, label="Generator Loss")
plt.plot (DD_loss, label="Discriminator Loss")
plt.legend()

Otra formaes modificar el tamaño de batch del discrimindor y del generador con las siguientes ecuaciones:

- disc_batch_size = int(batch/(d_loss[0]+1e-8))
- gen_batch_size =  int(batch/(g_loss+1e-8))

Notar que el termino : +1e-8 se utiliza para evitar la división entre cero.

In [ ]:
model_gen = Sequential()
model_gen.add (Dense (256, input_shape = (100,)))
model_gen.add (LeakyReLU (alpha = 0.2))
model_gen.add (BatchNormalization (momentum = 0.8))
model_gen.add (Dense (512))
model_gen.add (LeakyReLU (alpha = 0.2))
model_gen.add (BatchNormalization (momentum = 0.8))
model_gen.add (Dense(1024))
model_gen.add (LeakyReLU (alpha=0.2))
model_gen.add (BatchNormalization (momentum = 0.8))
model_gen.add (Dense (np.prod (in_shape), activation = 'tanh'))
model_gen.add (Reshape (in_shape))
model_gen.summary ()
model_gen.compile (loss='binary_crossentropy', optimizer=OPTIMZADOR_ADAM)

model_Disc = Sequential ()
model_Disc.add (Flatten(input_shape = in_shape))
model_Disc.add (Dense(128, input_shape = in_shape))
model_Disc.add (LeakyReLU(alpha = 0.2))
model_Disc.add (Dense(64))
model_Disc.add (LeakyReLU(alpha = 0.2))
model_Disc.add (Dense(1, activation = 'sigmoid'))
 
model_Disc.compile (loss='binary_crossentropy', optimizer=OPTIMZADOR_ADAM, metrics = ['accuracy'])

model_gan = Sequential()
model_gan.add (model_gen)
model_gan.add (model_Disc)
model_gan.compile (loss='binary_crossentropy', optimizer=OPTIMZADOR_ADAM, run_eagerly = True)


model_gan.summary()

In [ ]:
# Parámetros del entrenamiento
epochs = 5000
batch = 10
gen_batch_size = batch
disc_batch_size = batch/2

sample_interval = 100
num_classes     = 10
latent_dim      = 100
d_loss_prev     = np.inf
g_loss_prev     = np.inf

DD_loss = np.zeros((epochs,))
GG_loss = np.zeros((epochs,))


for cnt in range(epochs):

    ## Entrenamos discriminador
        # Imágenes reales
    random_index = np.random.randint(0, len(X_train) - np.int64(disc_batch_size))
    legit_images = X_train[random_index : random_index + np.int64(disc_batch_size)].reshape((np.int64(disc_batch_size),)+in_shape)
        # Imágenes sintéticas
    gen_noise = np.random.normal(0, 1, (np.int64(disc_batch_size), 100))
    print('gen_noise:',gen_noise.shape)
    syntetic_images = model_gen.predict(gen_noise)
        # Combinamos imágenes reales y sintéticas
    x_combined_batch = np.concatenate((legit_images, syntetic_images))
    y_combined_batch = np.concatenate((np.ones((np.int64(disc_batch_size), 1)), np.zeros((np.int64(disc_batch_size), 1))))
        # Entrenamos discriminador
    d_loss = model_Disc.train_on_batch(x_combined_batch, y_combined_batch)
   
    # Calculamos el cambio en la loss del discriminador
    print('d_loss_prev:',d_loss_prev)
    print('d_loss[0]:',d_loss[0])
 
    d_loss_prev = d_loss[0]
    disc_batch_size = int(batch/(d_loss[0]+1e-8))
         
    print('disc_batch_size:',disc_batch_size)
    ## Entrenamos generador
        # Imágenes sintéticas
    noise = np.random.normal(0, 1, (gen_batch_size, 100))
    y_mislabled = np.ones((gen_batch_size, 1))
        # Entremaos generador
    g_loss = model_gan.train_on_batch(noise, y_mislabled)
    
    gen_batch_size =  int(batch/(g_loss+1e-8))
  
    ## Evolución entrenamiento
    print ('epoch: %d, [Discriminator :: d_loss: %f], [ Generator :: loss: %f]' % (cnt, d_loss[0], g_loss))

    DD_loss[cnt] = d_loss[0]
    GG_loss[cnt] = g_loss

    if cnt % sample_interval == 0:

        # Generar imágenes falsas a partir de vectores de ruido aleatorios
        noise = np.random.normal(0, 1, (num_classes, latent_dim))
        images = model_gen.predict(noise)
        print('images:',images.shape)
             
        images = 0.5 * images + 0.5 # escalar de [-1,1] a [0,1]
        images = np.uint8(255 * images) # escalar a [0, 255]
        # Mostrar las imágenes generadas
        r = 2
        c = 5
        fig, axs = plt.subplots(r, c,figsize=(11,6.9),dpi=100)
        count = 0
        for i in range(r):
            for j in range(c):
                axs[i,j].imshow(images[count, :, :, 0], cmap='gray') #, cmap='gray'
                axs[i,j].axis('off') 
                count += 1
        plt.show()
     

In [ ]:
%matplotlib inline 
plt.figure
plt.plot (GG_loss, label="Generator Loss")
plt.plot (DD_loss, label="Discriminator Loss")
plt.legend()

# Generar Datos

In [ ]:
# Generamos imagen sintética
gen_noise = np.random.normal(0, 1, (np.int64(10), 100))
syntetic_images = model_gen.predict(gen_noise)

In [ ]:
# Mostramos imagen sintética
plt.imshow(syntetic_images[4,:,:,0],cmap='gray')
plt.axis('off')

In [ ]:
# Esto es una imagen real (por comparar)
plt.imshow(X_train[0,:,:,0],cmap='gray')
plt.axis('off')

### Ejercicio EXTRA:
Crea un modelo que incluya todas las modificaciones de los cuatro ejercicios anteriores.

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import tensorflow as tf 

from tensorflow.keras import initializers
from keras.layers import LeakyReLU
from keras.layers import Input, Dense, Reshape, Flatten, Dropout, Multiply
from keras.layers import BatchNormalization, Activation, Embedding, ZeroPadding2D
from keras.layers import MaxPooling2D, concatenate
from tensorflow.keras.utils import plot_model
from keras.layers.convolutional import UpSampling2D, Conv2D, Conv2DTranspose

from keras.models import Sequential, Model
from keras.optimizers import Adam

from keras.datasets import cifar10

In [ ]:
# Visualizar imagenes-labels  
def show_real_img (real_img,real_label,class_names,num_classes):
    real_img = ((real_img + 1) * 127.5).astype(np.uint8)
    fig, axes = plt.subplots (2, 5, figsize = (8, 3))
    axes = axes.flatten ()
    for i in range (num_classes):
        idx = np.where (real_label [:] == i)[0]
        features_idx = real_img [idx, ::]
        img_num = np.random.randint (features_idx.shape [0])
        img = features_idx[img_num, ::]
 
        axes [i].imshow (img [:,:,::-1]) #img [:,:,::-1].reshape(28, 28), cmap='gray')
        axes [i].set_title (class_names [i])   
        axes [i].axis ('off')   
    plt.tight_layout ()
    plt.show ()

In [ ]:
# Definir las variables relevantes
# Datos
# Load cifar10  
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

print('X_train shape:', X_train.shape)
print(X_train.shape[0], 'train samples')

num_classes = len(np.unique(y_train))
class_names = ['airplane','automobile','bird','cat','deer','dog',
                'frog','horse','ship','truck']

# Rescale -1 to 1
X_train = (X_train.astype(np.float32) - 127.5) / 127.5
 
# Normalizamos a 0-255
# X_train = X_train.astype('float32') / 255.0

# Visualizar imagenes cargadas del dataset
show_real_img (X_train,y_train,class_names,num_classes)

print('X_train shape 2:', X_train.shape)

In [ ]:
idx = np.array([3, 4, 0, 7, 2, 3, 0, 4, 8, 3])
np.array(class_names)[idx] 

In [ ]:
#----------------------------------------
# Datos para el modelo y el entrenamiento
#----------------------------------------

# Definir dimensiones de entrada
in_shape   = (32, 32, 3)
img_dim    = np.prod (in_shape)
init       = initializers.RandomNormal (stddev = 0.02)
latent_dim = 100
 
#optimizador
OPTIMZADOR_ADAM = Adam (learning_rate = 0.0002, beta_1 = 0.5)

In [ ]:
# ***********************************
# Definir modelo generador
# ***********************************
generator = Sequential ()
## Input layer  
generator.add(Dense(256, input_shape=(latent_dim,))) # , kernel_initializer=init
generator.add(BatchNormalization( momentum=0.8)) # momentum=0.8
generator.add(LeakyReLU(alpha=0.2))


generator.add(Dense(128 )) # , kernel_initializer=init
generator.add(BatchNormalization( momentum=0.8)) # momentum=0.8
generator.add(LeakyReLU(alpha=0.2))


generator.add(Dense(128 )) # , kernel_initializer=init
generator.add(BatchNormalization( momentum=0.8)) # momentum=0.8
generator.add(LeakyReLU(alpha=0.2))


generator.add(Dense(64))
generator.add(BatchNormalization()) # momentum=0.8
generator.add(LeakyReLU(alpha=0.2))

 
generator.add(Dense(np.prod(in_shape), activation='tanh'))
generator.add(Reshape(in_shape))

# se pasa también la eiqueta de la imagen
# Create label embeddings, one-hot (label)
label = Input (shape = (1,), dtype = 'int32')
label_embedding = Embedding (num_classes, latent_dim)(label)
label_embedding = Flatten ()(label_embedding)

# latent space
z = Input (shape = (latent_dim,))

# Output image
img = generator (Multiply ()([z, label_embedding]))

# Generator with condition input
generator = Model ([z, label], img)
generator.summary ()
plot_model (generator, to_file = 'cgan_generator.png', show_shapes = True)

# ***********************************
# Definir modelo discriminador
# ***********************************

# Discriminator network
discriminator = Sequential()
 
# Input layer  
 
# Input layer  
discriminator.add(Flatten(input_shape=in_shape))
discriminator.add(Dense(64, input_shape=in_shape)) # , kernel_initializer=init
discriminator.add(LeakyReLU(alpha=0.2))
#discriminator.add(BatchNormalization()) # momentum=0.8
#discriminator.add(Dropout(0.2))
discriminator.add(Dense(128)) # , kernel_initializer=init
discriminator.add(LeakyReLU(alpha=0.2))
#discriminator.add(BatchNormalization()) # momentum=0.8
#discriminator.add(Dropout(0.2))
discriminator.add(Dense(128)) # , kernel_initializer=init
discriminator.add(LeakyReLU(alpha=0.2))
#discriminator.add(BatchNormalization()) # momentum=0.8
#discriminator.add(Dropout(0.2))
discriminator.add(Dense(256 )) # , kernel_initializer=init
discriminator.add(LeakyReLU(alpha=0.2))
#discriminator.add(BatchNormalization()) # momentum=0.8
discriminator.add(Dropout(0.3))
# Output layer
 
discriminator.add(Dense(1, activation='sigmoid'))

# se pasa también la eiqueta de la imagen
label_d = Input(shape = (1,), dtype = 'int32')
label_embedding_d = Embedding (num_classes, img_dim)(label_d)
label_embedding_d = Flatten ()(label_embedding_d)
label_embedding_d = Reshape (in_shape)(label_embedding_d)
 
# image dimension 32x32
# Image input
img_d = Input (shape = in_shape)
# Output image
validity = discriminator (Multiply()([img_d, label_embedding_d]))

# Discriminator with condition input
discriminator = Model([img_d, label_d], validity)
discriminator.summary()
plot_model(discriminator, to_file='cgan_discriminator.png', show_shapes=True)
# Compilar modelos
discriminator.compile(loss='binary_crossentropy', optimizer=OPTIMZADOR_ADAM, metrics=['accuracy'])
#generator.compile(loss='binary_crossentropy', optimizer=OPTIMZADOR_ADAM)

# ***********************************
# Definir modelo cGan
# ***********************************
gen_input = Input (shape = (latent_dim,))
gen_label = Input (shape = (1,), dtype='int32')

synthetic_image = generator ([gen_input, gen_label])
synthetic_image_reshaped = Reshape (in_shape)(synthetic_image)

discriminator.trainable = False

validity = discriminator ([synthetic_image_reshaped, gen_label])

cGan = Model ([gen_input, gen_label], validity)
cGan.summary ()
plot_model (cGan, to_file = 'cgan_model.png', show_shapes = True)
cGan.compile (loss = 'binary_crossentropy', optimizer = OPTIMZADOR_ADAM, metrics = ['accuracy'])

In [ ]:
# Parámetros del entrenamiento
epochs     = 20000 
batch_size = 128

# Bucle entrenamiento
DD_loss = np.zeros ((epochs+1,))
GG_loss = np.zeros ((epochs+1,))
 
ones  = np.ones ((batch_size, 1))
zeros = np.zeros ((batch_size,1))

for cnt in range (epochs + 1):
   
    # ***********************************
    # Paso 1: Entrenar el discriminador
    discriminator.trainable = True
    # ***********************************
    
    
    # Genera un conjunto de imágenes reales
    idx = np.random.randint (0, X_train.shape [0], size = batch_size)
    #idx0 = cnt*batch_size
    #idx1 = (cnt+1)*batch_size

    real_images = X_train [idx] # batch_size imageners normalizadas entre [-1,1]
    real_labels = y_train [idx] #y_train[idx0:idx1] 

    # Genera un conjunto de imágenes falsas
    noise = np.random.normal (0, 1, (batch_size, latent_dim)) 
    fake_labels = np.random.randint (0, num_classes, batch_size).reshape (-1,1) 
    fake_images = generator.predict ([noise,fake_labels])
    #print(fake_images.shape)
    fake_images = np.reshape (fake_images, (fake_images.shape [0], 32, 32, 3))
    
    # Entrenamiento discriminador
    
    d_loss_real = discriminator.train_on_batch ([real_images, real_labels], ones)
    d_loss_fake = discriminator.train_on_batch ([fake_images, fake_labels], zeros)
    d_loss = 0.5 * np.add (d_loss_real, d_loss_fake)

    # ***********************************
    # Paso 2: Entrenar el generador
    discriminator.trainable = False
    # ***********************************
    gen_input  = np.random.normal (0, 1, (batch_size, latent_dim))
    gen_labels = np.random.randint (0, num_classes, batch_size).reshape (-1,1)  
    
    g_loss = cGan.train_on_batch ([gen_input, gen_labels], ones)

    # Registrar el progreso del entrenamiento
    DD_loss[cnt] = d_loss [0]
    GG_loss[cnt] = g_loss [0]

    print(f'Epoch: {cnt}/{epochs} \t Discriminator Loss: {d_loss[0]} \t Generator Loss: {g_loss[0]}')
    # Imprimir progreso cada 100 iteraciones
    if cnt % 100 == 0:
        print (y_train.shape)
         # Rescale images 0 - 1
        real_images = 0.5 * real_images + 0.5
        real_images = (real_images* 255).astype(np.uint8)
        #show_real_img (real_images, real_labels, class_names, num_classes)
        
        samples = 10
        z = np.random.normal(loc=0, scale=1, size=(samples, latent_dim))
        
        labels = np.arange(0, num_classes).reshape (-1,1) 
        

        x_fake = generator.predict([z, labels])
        x_fake =  np.reshape (x_fake, (x_fake.shape[0], 32, 32, 3))
        show_real_img (x_fake, labels, class_names, num_classes)


 

In [ ]:
%matplotlib inline
plt.figure
plt.plot(DD_loss, label="Discriminator Loss")
plt.plot(GG_loss, label="Generator Loss")
plt.legend()

In [ ]:
# Generar imágenes con etiquetas aleatorias
n_to_show   = 10
latent_dim  = 100
num_classes = 10

z = np.random.normal (loc = 0, scale = 1, size = (n_to_show, latent_dim))
fake_labels = np.random.randint (0, num_classes, n_to_show).reshape (-1,1)
x_fake = generator.predict ([z, fake_labels])
x_fake =  np.reshape(x_fake, (x_fake.shape [0], 32, 32, 3))
x_fake = 0.5 * x_fake + 0.5 # escalar de [-1,1] a [0,1]
x_fake = np.uint8(255 * x_fake).astype(np.uint8) # escalar a [0, 255]

# Mostrar imágenes generadas con sus etiquetas
fig, axs = plt.subplots(1, n_to_show, figsize=(20,6.9),dpi=100)
for i in range(n_to_show):
    axs[i].set_title(f'Label: {class_names[i]}')
    axs[i].imshow(x_fake[i, :,:])
    axs[i].axis('off')
plt.show() 

Volvemos a ver que exclusivamente con capas densas, y aún incluyendo más información como es el label de la imagen, la red no es capaz de acercarse a la imagen real.

# Con capas convolucionales 

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import tensorflow as tf 

from keras import initializers
from keras.layers import LeakyReLU
from keras.layers import Input, Dense, Reshape, Flatten, Dropout, Multiply, concatenate
from keras.layers import BatchNormalization, Activation, Embedding, ZeroPadding2D
from keras.layers import MaxPooling2D, concatenate, MaxPool2D
from tensorflow.keras.utils import plot_model
from keras.layers.convolutional import UpSampling2D, Conv2D, Conv2DTranspose

from keras.models import Sequential, Model
from keras.optimizers import Adam

from keras.datasets import cifar10

In [ ]:
#----------------------------------------
# Datos para el modelo y el entrenamiento
#----------------------------------------

# Definir dimensiones de entrada
in_shape          = (32, 32, 3)
img_dim           = np.prod (in_shape)
init              = initializers.RandomNormal (stddev = 0.02)
x_init            = tf.keras.initializers.GlorotUniform ()
x2_init           = tf.keras.initializers.GlorotNormal ()
OPTIMZADOR_ADAM   = Adam (learning_rate = 0.0002, beta_1 = 0.5)# learning_rate=0.0005, 
OPTIMZADOR_RMSPRO =  tf.keras.optimizers.experimental.RMSprop (learning_rate = 0.002)

EPOCHS        = 2
batch_size    = 256
latent_dim    = 100
embedding_dim = 28 #50

In [ ]:
# Definir las variables relevantes
# Datos
# Load cifar10  
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

print('X_train shape:', X_train.shape)
print(X_train.shape[0], 'train samples')

num_classes = len(np.unique(y_train))
class_names = ['airplane','automobile','bird','cat','deer','dog',
                        'frog','horse','ship','truck']



# Rescale -1 to 1
X_train, X_test = X_train.astype('float32')/127.5 - 1, X_test.astype('float32')/127.5 - 1


In [ ]:
# Visualizar imagenes-labels  
def show_real_img_rand (real_img,real_label,class_names,num_classes):
    print("show_real_img_rand")

    fig, axes = plt.subplots (2, 5, figsize = (8, 3))
    axes = axes.flatten ()
    for i in range (num_classes):
        idx = np.where (real_label [:] == i)[0]                # buscamos label 'i'
        features_idx = real_img [idx, ::]                      # buscamos las posiciones de las imagenes del label 'i'
        img_num = np.random.randint (features_idx.shape [0])   # cogemos el primer batch q tiene esa posición
    
        img = features_idx[img_num, ::]

        img = (((img) + 1) * 127.5).astype(np.uint8)

        axes [i].imshow (img [:,:,::-1]) #img [:,:,::-1]
        axes [i].set_title (class_names [i])   
        #axes [i].axis ('off')   
    plt.tight_layout ()
    plt.show ()
    
def show_real_img (real_img, real_label, class_names, num_classes):
    print ("*> show_real_img: ")
    lbls_img  = real_label [0:num_classes].ravel()
    names_img = [class_names [i] for i in  real_label [0:num_classes].ravel()]
    fig, axes = plt.subplots (2, 5, figsize = (8, 3))
    axes = axes.flatten ()
    for i  in range  (num_classes):
        img = ((real_img [i,:,:,::-1] + 1) * 127.5).astype(np.uint8)  
        axes [i].imshow (img)  
        axes [i].set_title (names_img[i])   
        #axes [i].axis ('off')   
    plt.tight_layout ()
    plt.show ()     
    
def show_predict_img (generator, class_names, num_classes, batch_size, latent_dim):
  print ("*> show_predict_img: " )
  try: 

    # Generar ruido y etiquetas aleatorias
 

    fake_label = np.random.randint(0, num_classes, size = batch_size).reshape (-1,1)
    
    lbls_img  = fake_label [0:num_classes].ravel()
    names_img = [class_names [i] for i in  fake_label [0:num_classes].ravel()]

    z = np.random.normal (loc = 0, scale = 1, size = (batch_size, latent_dim))
    z = np.clip (z, -1, 1)
    print (z.shape)
    print (fake_label.shape)
    x_fake = generator.predict ([z, fake_label])

    fig, axes = plt.subplots (2, 5, figsize = (8, 3))
    axes = axes.flatten ()
    for i  in range  (num_classes):
        img = ((x_fake [i,:,:,::-1] + 1) * 127.5).astype(np.uint8)  
        axes [i].imshow (img)  
        axes [i].set_title (names_img [i])   
        axes [i].axis ('off')   
    plt.tight_layout ()
    plt.show ()

  except Exception as inst:
    print(type(inst))    # the exception type
    print(inst.args)     # arguments stored in .args
    print(inst)        
    x = inst.args     # unpack args
    print('x =', x)

In [ ]:
### # define the standalone generator model
 
# Función para crear el generador
def create_generator(num_classes, latent_dim = 100,  embedding_dim = 28):
    # Entrada de ruido y label
   
    noise = Input(shape=(latent_dim,))
    in_label = Input(shape=(1,))
    le = Flatten()(Embedding(num_classes, embedding_dim)(in_label))
 
    m_nn      = 256
    n_nodes   = m_nn * 4 * 4
    dim_nodes = 4, 4, m_nn 
    # Concatenar ruido y etiqueta
    inputs = concatenate ([noise, le])

    # Creo la red del generador
    x = inputs
     
    #x = Dense (m_nn)(x)
    # Capa completamente conectada
    x = Dense (n_nodes)(x)
    x = Reshape (dim_nodes)(x)
    
    # Capas de deconvolución 8x8
    x = Conv2DTranspose (128, (4, 4), strides=(2, 2), padding='same')(x)
    #x = BatchNormalization ()(x) #momentum = 0.9
    #x = MaxPool2D ( )(x)
    x = LeakyReLU (alpha = 0.2)(x)
    
    # Capas de deconvolución 16x16
    x = Conv2DTranspose (128 , (4, 4), strides = (2,2), padding = 'same')(x)
    #x = BatchNormalization ( )(x) #momentum = 0.9
    #x = MaxPool2D ( )(x)
    x = LeakyReLU (alpha = 0.2)(x)

    # Capas de deconvolución 32x32
    x = Conv2DTranspose (128 , (4, 4), strides = (2, 2), padding = 'same')(x)
    #x = BatchNormalization (momentum = 0.9 )(x) # momentum = 0.9
    #x = MaxPool2D ( )(x)
    x = LeakyReLU (alpha = 0.2)(x)
    
    
    #x = Conv2DTranspose (256  , (5, 5), strides = (2, 2), padding = 'same')(x)
    #x = BatchNormalization (momentum = 0.8)(x)
    #x = MaxPool2D ( )(x)
    #x = LeakyReLU (alpha = 0.2)(x)
  
        
    # Capa de salida
    output = Conv2DTranspose (3, (3, 3), strides = (1, 1), padding = 'same', activation = 'tanh')(x)
    
    # Crear modelo
    generator = Model([noise, in_label], output)
    generator.compile (loss = 'binary_crossentropy', optimizer = Adam (learning_rate = 0.002, beta_1 = 0.5) )

    return generator

# Crear generador
 
generator = create_generator(latent_dim = latent_dim, num_classes = num_classes)
 
generator.summary()
    

# Función para construir el discriminador condicional
def define_discriminator (in_shape = (32, 32, 3), n_classes = 10):
    in_label =  Input (shape = (1,))
    
    le = Embedding (num_classes, img_dim)(in_label)
    le = Flatten ()(le)
    le = Reshape (in_shape)(le)
    
    in_image =  Input (shape =  in_shape)
    merge    =  concatenate ([in_image, le])
     
 
    x = Dense (64, kernel_initializer = init) (merge)
   
    # downsample
    fe = Conv2D (128 , (3, 3), strides = (2, 2), padding = 'same')(x)
    fe = LeakyReLU (alpha = 0.2)(fe)
    #fe = Dropout (0.4)(fe)
    
    # downsample
    fe = Conv2D (128 , (3, 3), strides = (2, 2), padding = 'same')(fe)
    fe = LeakyReLU (alpha = 0.2)(fe)
    #fe = Dropout (0.4)(fe)
     
    
    # downsample
    fe = Conv2D (128 , (3, 3), strides = (2, 2), padding = 'same')(fe)
    fe = LeakyReLU (alpha = 0.2)(fe)
    #fe = Dropout (0.4)(fe)
   
    
    # downsample
    fe = Conv2D (128 , (3, 3), strides = (2, 2), padding = 'same')(fe)
    fe = LeakyReLU (alpha = 0.2)(fe)
    fe = Dropout (0.4)(fe)
    fe = Flatten ()(fe)
 
    fe = Dense (256, activation = 'relu')(fe)
    fe = Dense (256, activation = 'relu')(fe)
    
    fe = Dropout (0.4)(fe)
        
    out_layer =  Dense (1, activation = 'sigmoid')(fe)
    model     =  Model ([in_image, in_label], out_layer)
    
    model.compile (loss = 'binary_crossentropy', optimizer = Adam (learning_rate = 0.0004, beta_1 = 0.5), metrics = ['accuracy'])  
    
    return model
 
discriminator = define_discriminator(in_shape, num_classes)
discriminator.summary()

#define the Conditional GAN
def define_gan (generator, discriminator):
    #make discriminator non trelue
    discriminator.trainable = False
    #get noise and label from generator
    gen_noise, gen_label = generator.input
 
    #get output from generator
    gen_output = generator.output
  
    #connect image and label input from generator as inputs to discriminator
    gan_output = discriminator ([gen_output, gen_label])
    #define the GAN model. 
  
    model =  tf.keras.Model ([gen_noise, gen_label], gan_output)
    #compile model
 
    model.compile (loss='binary_crossentropy', optimizer = Adam (learning_rate = 0.0005, beta_1 = 0.5), metrics = ['accuracy'])
    return model

cGan = define_gan(generator, discriminator)
cGan.summary()

def get_real_images (batch_size):
    print ("REAL _IMAGENES")
    #(X_train, y_train), (_, _) = tf.keras.datasets.cifar10.load_data()
    # Rescale -1 to 1
    #X_train = (X_train.astype(np.float32) - 127.5) / 127.5
    idx = np.random.randint (0, X_train.shape[0], batch_size)
    real_images = X_train [idx]
    real_label  = y_train [idx].reshape (-1, 1) 
    
    #show_real_img_ (real_images,real_label,class_names,num_classes)
    
    return real_images, real_label

def get_fake_labels (batch_size):
    """
    Genera etiquetas falsas para las imágenes falsas generadas por el generador del modelo GAN.

    Parameters:
        - batch_size (int): Tamaño del batch de imágenes a generar.

    Returns:
        - fake_labels (array): Array de etiquetas falsas generadas para las imágenes falsas.
    """
    fake_labels = np.random.randint (0, num_classes, batch_size) 
    #labels_one_hot = to_categorical(fake_labels, num_classes = num_classes)
    return fake_labels.reshape (-1, 1) 
# Función para generar un batch de imágenes falsas junto con sus labels
def get_fake_images (batch_size, latent_dim):
    """
    Genera imágenes falsas utilizando el generador del modelo GAN.

    Parameters:
        - batch_size (int): Tamaño del batch de imágenes a generar.
        - latent_dim (int): Dimensión del espacio latente.

    Returns:
        - fake_images (array): Array de imágenes falsas generadas por el generador.
        - fake_labels (array): Array de etiquetas falsas generadas para las imágenes falsas.
    """
 
    print ("FAKE_IMAGENES")

    # Generar ruido y etiquetas aleatorias
    noise  = np.random.normal(0, 1, (batch_size, latent_dim))
    #noise  = (noise.astype(np.float32) - 127.5) / 127.5
    noise = np.clip (noise, -1, 1)
    fake_labels = get_fake_labels (batch_size)
    # Llamar al generador para generar imágenes. los valores de píxeles de encuentran entre (-1, 1). recordar que la f.Actv es tanh
    fake_images = generator.predict([noise, fake_labels])
    # Escalar los valores de píxeles a (0, 255)
    #fake_images = 127.5 * generated_images + 127.5
 
    return fake_images, fake_labels



In [ ]:
show_real_img_rand (X_train, y_train, class_names, num_classes)

show_real_img (X_train, y_train, class_names, num_classes)

show_predict_img (generator, class_names, num_classes, num_classes, latent_dim)

## ***TRAIN***

1. Definimos las variables "gen_batch_size" y "disc_batch_size" que serán las que determinen el tamaño del batch de cada modelo en cada iteración del bucle de entrenamiento. Establecemos su valor inicial en "batch", el tamaño de batch original.

3. En cada iteración del bucle de entrenamiento, calculamos el coste (loss) de cada modelo en la iteración anterior y lo almacenamos en las variables "d_loss_prev" y "g_loss_prev".

3. Para el discriminador, comparamos su coste anterior con el coste actual y, si ha mejorado, reducimos el tamaño del batch en un 10%, mientras que si ha empeorado, lo aumentamos en un 10%. En cualquier caso, aseguramos que el tamaño mínimo del batch sea 1.

4. Para el generador, hacemos lo mismo que para el discriminador pero con la inversa, es decir, si ha mejorado aumentamos el tamaño del batch y si ha empeorado lo reducimos.

5. Utilizamos los tamaños de batch calculados para entrenar los modelos en cada iteración.

In [ ]:
EPOCHS        = 100
batch_size    = 256
latent_dim    = 100
embedding_dim = 28
smooth        = 0.1

real = np.ones  (shape = (batch_size, 1))
fake = np.zeros (shape = (batch_size, 1))

d_loss = []
g_loss = []
DD_loss  = []
GG_loss  = []
for e in range(EPOCHS + 1):
    for i in range(len(X_train) // batch_size):
        # Paso 1: Entrenar el discriminador
        discriminator.trainable = True

        # Genera un conjunto de imágenes reales
        idx = np.random.randint(0, X_train.shape[0], size=batch_size)
        real_images = X_train[idx]
        real_labels = y_train[idx]

        # Genera un conjunto de imágenes falsas
        noise = np.random.normal(0, 1, (batch_size, latent_dim))
        noise = np.clip(noise, -1, 1)
        fake_labels = np.random.randint(0, num_classes, batch_size)
        fake_images = generator.predict([noise, fake_labels])

        d_loss_real = discriminator.train_on_batch([real_images, real_labels], real * (1 - smooth))
        d_loss_fake = discriminator.train_on_batch([fake_images, fake_labels], fake)
        d_loss_batch = 0.5 * np.add(d_loss_real, d_loss_fake)
        d_loss.append(d_loss_batch)

        # Paso 2: Entrenar el generador
        discriminator.trainable = False

        gen_input = np.random.normal(0, 1, (batch_size, latent_dim))
        gen_input = np.clip(gen_input, -1, 1)
        gen_labels = np.random.randint(0, num_classes, batch_size)

        g_loss_batch = cGan.train_on_batch([gen_input, gen_labels], real)
        g_loss.append(g_loss_batch)

    # Registrar el progreso del entrenamiento
    DD_loss.append(np.mean(d_loss, axis=0))
    GG_loss.append(np.mean(g_loss, axis=0))
    print(f'Epoch: {e}/{EPOCHS} \t Discriminator Loss: {DD_loss[0]} \t Generator Loss: {GG_loss[0]}')

    # Reiniciar las listas de pérdida para la siguiente época
    d_loss = []
    g_loss = []

    # Imprimir progreso cada 10 iteraciones
    #if e % 5 == 0:
    show_predict_img (generator, class_names, num_classes, num_classes, latent_dim)


In [ ]:
### %matplotlib inline
plt.figure ()
plt.plot (DD_loss , label = "Discriminator")
plt.plot (GG_loss  , label = "Generator")
plt.legend ()

In [ ]:
# Generar imágenes con etiquetas aleatorias
n_to_show   = 15
latent_dim  = 100
num_classes = 10
show_predict_img (generator, class_names, num_classes, n_to_show, latent_dim)

#### Pruebas de GPU

In [ ]:
import tensorflow as tf
print(tf.config.list_physical_devices('GPU'))

In [ ]:
import tensorflow as tf 

if tf.config.list_physical_devices('GPU'):  # GPUが使えたら利用する
    device_name = tf.test.gpu_device_name()
else:
    device_name = '/CPU:0'
    
print(device_name)

In [ ]:

import tensorflow as tf

# Limitación la memoria de la GPU
config = tf.compat.v1.ConfigProto(allow_soft_placement=True)
config.gpu_options.per_process_gpu_memory_fraction = 0.6
tf.compat.v1.keras.backend.set_session (tf.compat.v1.Session(config=config))

# Permitir crecimiento de la memoria
physical_devices = tf.config.list_physical_devices('GPU')
try:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)
except:
    print('Invalid device or cannot modify virtual devices once initialized.')


print('#### INFORMACIÓN ####')
print('  Versión de TensorFlow: {}'.format(tf.__version__))
#print('  GPU: {}'.format([(x.name, x.physical_device_desc)
#                          for x in tf.python.client.device_lib.list_local_devices() if x.device_type == 'GPU']))
print( tf.sysconfig.get_build_info() )
 

In [ ]:
import tensorflow as tf
print(tf.__version__)

In [ ]:
 
import tensorflow as tf

gpus = tf.config.list_physical_devices('GPU')
if gpus:
    print("TensorFlow está usando GPU.")
else:
    print("TensorFlow está usando CPU.")

In [ ]:
import tensorflow as tf

gpus = tf.config.list_physical_devices('GPU')
if gpus:
  try:
    # Ajustar la asignación de memoria de TensorFlow para todas las GPUs disponibles en el sistema
    for gpu in gpus:
        tf.config.experimental.set_memory_growth(gpu, True)
  except RuntimeError as e:
    # Manejar error
    print(e)

In [ ]:
import pkg_resources
libs = ["protobuf","tfds-nightly","googleapis-common-protos", "mediapipe",
        "tb-nightly", "tensorboard", "tensorflow", "tensorflow-gpu","tensorflow-intel", "tensorflow-metadata", 
        "tf_nightly_intel"]

[lb+' = '+pkg_resources.get_distribution(lb).version for lb in libs] 

In [ ]:
import os
os.environ["PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION"] = "python"

In [ ]:
import os

# Imprimir todas las variables de entorno
#print(os.environ)

# Acceder a una variable de entorno específica
valor_variable = os.environ.get ('CUDA_HOME')
print(valor_variable)
valor_variable = os.environ.get ('PATH')
print(valor_variable)

In [ ]:
import tensorflow as tf
tf.sysconfig.get_build_info()

In [ ]:
import tensorflow as tf
print(tf.config.set_visible_devices(tf.config.list_physical_devices('GPU'), 'GPU'))